In [1]:
import os
import numpy as np
import pandas as pd
import collections
import matplotlib.pyplot as plt
import json
#import mpl_stylesheet
import utils
#mpl_stylesheet.banskt_presentation(fontfamily = 'latex-clearsans', fontsize = 14, 
#                                   colors = 'banskt', dpi = 300, black = '#9ea8ad')

ModuleNotFoundError: No module named 'utils'

In [ ]:
tissue_file = '../metadata//tissue_table.txt'
json_file = '../metadata/gtex_v8_metadata.json'
nsample_file = '../metadata/tissue_nsamples.txt'

tshorts, tfulls, tstrings = utils.read_tissues(tissue_file)
with open(json_file) as instream:
    gtex_meta = json.load(instream)
tissue_colors = dict()
tissue_names = dict()
tissue_nsamples = dict()

for tshort, tfull, tstring in zip(tshorts, tfulls, tstrings):
    if tshort in tshorts:
        tissue_names[tshort] = tstring
        tissue_colors[tshort] = "#" + gtex_meta[tfull]["colorHex"]
        
tissue_nsamples = dict()
with open(nsample_file, 'r') as instream:
    for line in instream:
        tshort = line.strip().split()[0].strip()
        tissue_nsamples[tshort] = int(line.strip().split()[1].strip())
        
brain_tissues = ['bam', 'ban', 'bca', 'bceh', 'bce', 'bco', 'bfr', 'bhi', 'bhy', 'bnu', 'bpu', 'bsp', 'bsu']
sb006_tissues = ['haa', 'pan', 'spl', 'wb']

In [3]:
OPTPATH_FIELDS = ['sbeta', 'ngp', 'qvar']
class OptPath(collections.namedtuple('_OptPath', OPTPATH_FIELDS)):
    __slots__ = ()
    
def read_sboptim_results(infile):
    sbpath = list()
    with open(infile, 'r') as instream:
        #sboptim = float(instream.readline().strip().split()[0])
        for line in instream:
            lsplit = line.strip().split('\t')
            sbeta = float(lsplit[0].strip())
            kurt = float(lsplit[1].strip())
            qvar = float(lsplit[2].strip())
            sbinstance = OptPath(sbeta = sbeta, ngp = abs(kurt), qvar = qvar)
            sbpath.append(sbinstance)
    return sbpath

def optimum_sigbeta(res, shift = 0.05):
    ngp = np.array([x.ngp for x in res])
    qvar = np.array([x.qvar for x in res])
    sbeta = np.array([x.sbeta for x in res])
    ngp_diff = ngp - np.min(ngp)
    min_sbeta = sbeta[np.argmin(ngp)]
    
    maskngp = ngp_diff <= shift
    maskqvr = qvar >= 0.005
    mask = np.logical_and(maskngp, maskqvr)
    if np.sum(mask) > 0:
        opt_sbeta = np.max(sbeta[mask])
    else:
        opt_sbeta = np.nan
        
    return opt_sbeta

In [ ]:
subtypes = ["tpms_EUR", ]
res = dict()
select_tissues = ["haa", "pan", "spl", "wb"]
for t in select_tissues:
    res[t] = dict()
    for st in subtypes:
        tfile = os.path.join(a2t_dir, f'{t}_{st}_sboptim.txt')
        if os.path.exists(tfile):
            res[t][st] = read_sboptim_results(tfile)

In [ ]:
# whichtissues = [x for x in tshorts if x in res.keys() and x not in sb006_tissues]
# fileprefix = f'gtex_alpha2t_sb01_tissues_{runtype}'
# subplot_h = 1.8

# whichtissues = [x for x in tshorts if x in res.keys()] # and x in sb006_tissues]
# fileprefix = f'gtex_alpha2t_sb006_tissues_{runtype}'
# subplot_h = 1.8

# bgcolor = '#F0F0F0'
# highlight_color = '#EE6868'
# highlight_crossmap_color = '#6887ED'
# highlight_crossmap_color2 = '#000000'
# highlight_colors = dict(zip(subtypes, [highlight_color, highlight_crossmap_color, highlight_crossmap_color2]))
# subdue_color = '#848f94'
# text_color = '#69767c'

# nplot = len(whichtissues)
# ncol  = 4
# nrow  = int(nplot / ncol + 1) if nplot%ncol != 0 else int(nplot / ncol)
# figw  = ncol * subplot_h + (ncol - 1) * 0.3 + 1.2
# figh  = nrow * subplot_h + (nrow - 1) * 0.3 + 1.5

In [ ]:
fig = plt.figure(figsize = (figw, figh))
axmain = fig.add_subplot(111)

for i, tshort in enumerate(whichtissues):
    ax = fig.add_subplot(nrow, ncol, i + 1)
    
    chosen_sb = 0.1 if tshort not in sb006_tissues else 0.006
    # Procedure explained below
    sbeta = [np.log10(x.sbeta) for x in res[tshort][subtypes[0]]]
    aprox_sbeta_ix = np.argmin(np.abs(np.array(sbeta) - np.log10(chosen_sb))) # find sbeta closest to 0.006
    ngp_sb0006 = res[tshort][subtypes[0]][aprox_sbeta_ix].ngp
    new_ngp_sb0006 = res[tshort][subtypes[1]][aprox_sbeta_ix]  #because sbeta seq is the same, get the ngp in the new curve
    
    new_ngps = [x.ngp for x in res[tshort][subtypes[1]]]
    aprox_ngp_ix = np.argmin(np.abs(np.array(new_ngps[aprox_sbeta_ix:]) - ngp_sb0006))
    if np.abs(res[tshort][subtypes[1]][aprox_sbeta_ix+aprox_ngp_ix].ngp - ngp_sb0006) > 0.005:
        aprox_ngp_ix = np.argmin(np.abs(np.array(new_ngps) - ngp_sb0006))
        new_sbeta = res[tshort][subtypes[1]][aprox_ngp_ix]
    else:
        new_sbeta = res[tshort][subtypes[1]][aprox_sbeta_ix+aprox_ngp_ix]
    print(f"{tshort}: {new_sbeta}")
    
    # Plot line for new optimal sbeta
    ax.axvline(np.log10(new_sbeta.sbeta), ymax = 0.6, color = "green", ls = 'dashed')

    
    for j, st in enumerate(subtypes):
        sbeta = [np.log10(x.sbeta) for x in res[tshort][st]]
        ngp   = [x.ngp for x in res[tshort][st]]
        ax.plot(sbeta, ngp, lw = 2, color = highlight_colors[st], zorder = 5)

    #     opt_sbeta = optimum_sigbeta(res[tshort], shift = 0.05)
    #     if not np.isnan(opt_sbeta):
    #         sbeta_choose_idx = np.argmin(np.array([abs(x - opt_sbeta) for x in sigbeta_choices]))
    #         tlist_sb[sbeta_choose_idx].append(tshort)
    #         ax.axvline(np.log10(sigbeta_choices[sbeta_choose_idx]), ymax = 0.6, color = subdue_color, ls = 'dashed')

        opt_sbeta = 0.1 if tshort not in sb006_tissues else 0.006
        ax.axvline(np.log10(opt_sbeta), ymax = 0.6, color = subdue_color, ls = 'dashed')
        
        cm_opt_sbeta = 0.01
        if tshort in sb006_tissues:
            ax.axvline(np.log10(cm_opt_sbeta), ymax = 0.6, color = "#626a6d", ls = 'dashed')


        ax.text(0.95, 0.95, tissue_names[tshort], va='top', ha='right', transform=ax.transAxes, color = text_color)
        ax.tick_params(bottom = False, top = False, left = False, right = False,
                       labelbottom = False, labeltop = False, labelleft = False, labelright = False)

        ax.set_facecolor(bgcolor)
        for side, border in ax.spines.items():
            border.set_visible(False)
        if i < ncol:
            ax.tick_params(top = True, labeltop = True, color = bgcolor, width = 5)
            ax.set_xticks(np.log10([0.001, 0.01, 0.1, 1.0]))
        if i%ncol == 0:
            ax.tick_params(left = True, labelleft = True, color = bgcolor, width = 5)
        ax.set_ylim(-0.1, 2.1)
    
axmain.tick_params(bottom = False, top = False, left = False, right = False,
                   labelbottom = False, labeltop = False, labelleft = False, labelright = False)
for side, border in axmain.spines.items():
    border.set_visible(False)
axmain.set_ylabel(r'$\alpha(\gamma)$', labelpad = 40, color = text_color)
axmain.set_xlabel('$\log_{10}(\gamma)$', labelpad = 50, color = text_color)
axmain.xaxis.set_label_position('top') 


plt.tight_layout()
# plt.savefig(f'../plots/{fileprefix}.pdf', bbox_inches='tight')
# plt.savefig(f'../plots/{fileprefix}.png', bbox_inches='tight')
plt.show()